In [67]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
from config import census_key
import json
import requests 
#from census import Census


In [68]:
# Import Accident dataset

accident_data = pd.read_csv(r'usaccidents.csv')

accident_data.head()


,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [69]:
# Drop unnecessary columns

reduced_accident_df = accident_data[["ID", "Severity", "Start_Time", "Start_Lat", "Start_Lng", "Street", "County", "State", "Zipcode", 
                                     "Weather_Timestamp", "Temperature(F)", "Wind_Chill(F)", "Humidity(%)", "Pressure(in)", 
                                     "Visibility(mi)", "Wind_Direction", "Wind_Speed(mph)", "Precipitation(in)", "Weather_Condition"]]

df = pd.DataFrame(reduced_accident_df)

df['Start_Time'] = pd.to_datetime(df['Start_Time'])

df['Start_Time'] = df['Start_Time'].apply(lambda x: x.date())


# Convert dates(object) to integers

df['Start_Time'] = pd.to_datetime(df['Start_Time'])


In [70]:
# Limit data to two-year period

start_date = "2017-01-01"
end_date = "2019-01-01"

mask = (df['Start_Time'] > start_date) & (df['Start_Time'] <= end_date)
two_year_df = df.loc[mask]
two_year_df

,ID,Severity,Start_Time,Start_Lat,Start_Lng,Street,County,State,Zipcode,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition
9217,A-9218,2,2017-01-02,38.234737,-122.664314,Bodega Ave,Sonoma,CA,94952-1748,2017-01-02 14:15:00,46.4,45.2,76.0,30.02,10.0,NNW,3.5,NaN,Overcast
9218,A-9219,2,2017-01-02,38.246796,-122.627808,E Washington St,Sonoma,CA,94952,2017-01-02 14:15:00,46.4,45.2,76.0,30.02,10.0,NNW,3.5,NaN,Overcast
9219,A-9220,2,2017-01-02,38.192196,-122.543053,Old Lakeville No 2 Rd,Sonoma,CA,94954,2017-01-02 14:35:00,46.4,45.2,81.0,30.03,10.0,NW,3.5,NaN,Overcast
9220,A-9221,3,2017-01-02,37.732292,-122.414909,I-280 S,San Francisco,CA,94110,2017-01-02 12:56:00,52.0,NaN,61.0,30.02,10.0,SSW,15.0,NaN,Overcast
9221,A-9222,3,2017-01-02,38.211773,-122.144753,I-80 E,Solano,CA,94534,2017-01-02 14:54:00,52.0,NaN,66.0,29.98,10.0,SW,12.7,NaN,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899386,A-2899410,2,2017-08-30,34.055160,-118.214070,San Bernardino Fwy E,Los Angeles,CA,90033,2017-08-30 19:47:00,82.0,NaN,49.0,29.68,10.0,Calm,NaN,NaN,Clear
2899387,A-2899411,2,2017-08-30,34.046740,-118.214800,I-5 N,Los Angeles,CA,90033,2017-08-30 19:47:00,82.0,NaN,49.0,29.68,10.0,Calm,NaN,NaN,Clear
2899388,A-2899412,3,2017-08-30,34.031850,-117.135910,I-10 E,San Bernardino,CA,92373,2017-08-30 19:50:00,98.6,NaN,19.0,29.75,6.0,SW,6.9,NaN,Haze
2899389,A-2899413,4,2017-08-30,34.277860,-118.608280,Ronald Reagan Fwy W,Los Angeles,CA,91311,2017-08-30 20:51:00,95.0,NaN,22.0,29.69,10.0,West,3.5,NaN,Clear


In [71]:
# convert date field to integer and use for join field
new_dates = two_year_df['Start_Time'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)

two_year_df['AccidentJoin'] = new_dates

# rename dataframe
accident_2_df = two_year_df

C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [72]:
# Get Holiday List
holidays = pd.read_csv(r'holidaylist.csv')

# filter down list to 2 years 
start_date = "2017-01-01"
end_date = "2019-01-01"

holiday_df = pd.DataFrame(holidays)

# convert Date to datetime field
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'])

# filter down dataframe
mask = (holiday_df['Date'] > start_date) & (holiday_df['Date'] <= end_date)
holiday_2_df = holiday_df.loc[mask]


In [73]:
#holiday_2_df

# convert date field to integer and use for join field
new_dates = holiday_2_df['Date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)

holiday_2_df['HolidayJoin'] = new_dates

holiday_2_df





C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,Date,Holiday,HolidayJoin
60,60,2017-01-02,New Year's Day,20170102
61,61,2017-01-16,"Birthday of Martin Luther King, Jr.",20170116
62,62,2017-02-20,Washington's Birthday,20170220
63,63,2017-05-29,Memorial Day,20170529
64,64,2017-07-04,Independence Day,20170704
65,65,2017-09-04,Labor Day,20170904
66,66,2017-10-09,Columbus Day,20171009
67,67,2017-11-10,Veterans Day,20171110
68,68,2017-11-23,Thanksgiving Day,20171123
69,69,2017-12-25,Christmas Day,20171225


In [74]:
#  Merge Accident and Holiday dataframes

accident_2_df.merge(holiday_2_df,how='left', left_on='AccidentJoin', right_on='HolidayJoin')

accident_2_df



,ID,Severity,Start_Time,Start_Lat,Start_Lng,Street,County,State,Zipcode,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,AccidentJoin
9217,A-9218,2,2017-01-02,38.234737,-122.664314,Bodega Ave,Sonoma,CA,94952-1748,2017-01-02 14:15:00,46.4,45.2,76.0,30.02,10.0,NNW,3.5,NaN,Overcast,20170102
9218,A-9219,2,2017-01-02,38.246796,-122.627808,E Washington St,Sonoma,CA,94952,2017-01-02 14:15:00,46.4,45.2,76.0,30.02,10.0,NNW,3.5,NaN,Overcast,20170102
9219,A-9220,2,2017-01-02,38.192196,-122.543053,Old Lakeville No 2 Rd,Sonoma,CA,94954,2017-01-02 14:35:00,46.4,45.2,81.0,30.03,10.0,NW,3.5,NaN,Overcast,20170102
9220,A-9221,3,2017-01-02,37.732292,-122.414909,I-280 S,San Francisco,CA,94110,2017-01-02 12:56:00,52.0,NaN,61.0,30.02,10.0,SSW,15.0,NaN,Overcast,20170102
9221,A-9222,3,2017-01-02,38.211773,-122.144753,I-80 E,Solano,CA,94534,2017-01-02 14:54:00,52.0,NaN,66.0,29.98,10.0,SW,12.7,NaN,Mostly Cloudy,20170102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899386,A-2899410,2,2017-08-30,34.055160,-118.214070,San Bernardino Fwy E,Los Angeles,CA,90033,2017-08-30 19:47:00,82.0,NaN,49.0,29.68,10.0,Calm,NaN,NaN,Clear,20170830
2899387,A-2899411,2,2017-08-30,34.046740,-118.214800,I-5 N,Los Angeles,CA,90033,2017-08-30 19:47:00,82.0,NaN,49.0,29.68,10.0,Calm,NaN,NaN,Clear,20170830
2899388,A-2899412,3,2017-08-30,34.031850,-117.135910,I-10 E,San Bernardino,CA,92373,2017-08-30 19:50:00,98.6,NaN,19.0,29.75,6.0,SW,6.9,NaN,Haze,20170830
2899389,A-2899413,4,2017-08-30,34.277860,-118.608280,Ronald Reagan Fwy W,Los Angeles,CA,91311,2017-08-30 20:51:00,95.0,NaN,22.0,29.69,10.0,West,3.5,NaN,Clear,20170830


In [19]:
# Census API

# State
url = "https://api.census.gov/data/2010/dec/sf1?get=P001001,NAME&for=state:*&key="

# Build query URL
query_url = url + census_key

census_response = requests.get(query_url)
census_json = census_response.json()
census_state_pd = pd.DataFrame(census_json)

# Drop column labels
census_state_df = census_state_pd.drop([census_state_pd.index[0]])

# drop unwanted column
census_state_df = census_state_df.drop(census_state_df.columns[[2]], axis=1)

# rename columns
census_state_df.columns = ['Pop', 'State']

# rearrange columns
census_state_df = census_state_df[['State', 'Pop']]

census_state_df


,State,Pop
1,Alabama,4779736
2,Alaska,710231
3,Arizona,6392017
4,Arkansas,2915918
5,California,37253956
6,Louisiana,4533372
7,Kentucky,4339367
8,Colorado,5029196
9,Connecticut,3574097
10,Delaware,897934


In [25]:
# County data for Ohio
url = "https://api.census.gov/data/2010/dec/sf1?get=P001001,NAME&for=county:*&key="

# Build query URL
query_url = url + census_key

census_response = requests.get(query_url)
census_json = census_response.json()

census_county_pd = pd.DataFrame(census_json)

# Split county name from state

# new data frame with split value columns 
new = census_county_pd[1].str.split(", ", n = 1, expand = True) 
  
# making separate county column from new data frame 
census_county_pd["County"]= new[0] 
  
# making separate state column from new data frame 
census_county_pd["State"]= new[1] 
  
# Dropping old Name columns 
census_county_pd.drop(columns =[1], inplace = True) 

# Drop column labels
census_county_pd = census_county_pd.drop([census_county_pd.index[0]])

# Drop extra columns
census_county_pd.drop(columns =[2], inplace = True)
census_county_pd.drop(columns =[3], inplace = True)

# filter down to state of ohio
ohio_county = census_county_pd.loc[(census_county_pd['State'] == "Ohio")]

# rename columns
ohio_county.columns = ['Pop', 'County', 'State']


# rearrange columns
ohio_county = ohio_county[['County', 'Pop']]

ohio_county
#census_county_pd

,County,Pop
1113,Jefferson County,69709
1114,Mahoning County,238823
1115,Lake County,230041
1116,Lawrence County,62450
1117,Licking County,166492
...,...,...
1761,Washington County,61778
1765,Wayne County,114520
1766,Williams County,37642
1767,Wood County,125488


In [30]:
# Zip
url = "https://api.census.gov/data/2010/dec/sf1?get=P001001,NAME&for=zip%20code%20tabulation%20area:*&key="

# Build query URL
query_url = url + census_key

census_response = requests.get(query_url)
census_json = census_response.json()

census_zip_pd = pd.DataFrame(census_json)

# Dropping old Name columns 
census_zip_pd.drop(columns =[1], inplace = True) 

# Drop column labels
ohio_zip = census_zip_pd.drop([census_zip_pd.index[0]])

# rename columns
ohio_zip.columns = ['Pop', 'Zip']

# rearrange columns
ohio_zip = ohio_zip[['Zip', 'Pop']]

# filter down to state of ohio
#ohio_zip = ohio_zip.loc[(ohio_zip['State'] == "Ohio")]

ohio_zip


,Zip,Pop
1,77477,35897
2,77478,24739
3,77479,74514
4,77480,8097
5,77481,35
...,...,...
33116,01841,47225
33117,01851,30190
33118,01852,33508
33119,01886,21951
